# Basic MPS/MPO operations
This notebook implements some of the basic MPS Arithmetic as well as MPO-action on an MPS.

In [1]:
import numpy as np
import opt_einsum as oe

We start with initializing two random vectors of length $2^n$.

In [2]:
n=6
vector_shape = 2**n
v = np.random.random(vector_shape)
w = np.random.random(vector_shape)

Reshape those vectors to have $n$ binary indices.

In [3]:

mps_shape = (2,)*n
v_reshaped = v.reshape(mps_shape)
w_reshaped = w.reshape(mps_shape)

## MPS Decomposition

The decomposition iterates three main steps:
* Reshape tensor as matrix with one index to the "right" and the rest as multiindex to the "left".
* Decompose that matrix via a (truncated) SVD.
* Contract the singular values with the right side

The result is stored in a List of 3rd order arrays. With the following shapes
$$[ (1, 2, \alpha_1), (\alpha_1, 2, \alpha_2), \dots, (\alpha_{n-1}, 2, 1) ],$$
where the $\alpha_i$ are the bond dimensions.

In [4]:
from utils.numpy_ops import truncated_svd

def MPS(tensor, max_bond=None):
    shape = tensor.shape
    MPS_tensors = []
    shape_right = 1
    for site in range(len(shape) - 1, 0, -1):
        shape_phys = shape[site]
        tensor_reshaped = tensor.reshape(-1, shape_phys * shape_right)                  # reshaping
        U, S, V = truncated_svd(tensor_reshaped, max_bond)                              # SVD
        tensor = U @ np.diag(S)                                                         # contracting the singular values
        shape_left = len(S)
        MPS_tensors = [V.reshape(shape_left, shape_phys, shape_right)] + MPS_tensors
        shape_right = shape_left
    MPS_tensors = [tensor.reshape(1, shape_phys, shape_right)] + MPS_tensors
    return MPS_tensors


We now use this function to decompose v_MPS and w_MPS

In [5]:
v_MPS = MPS(v_reshaped)
w_MPS = MPS(w_reshaped)

print("shapes of the tensors that make up v_MPS:")
print([core.shape for core in v_MPS])

shapes of the tensors that make up v_MPS:
[(1, 2, 2), (2, 2, 4), (4, 2, 8), (8, 2, 4), (4, 2, 2), (2, 2, 1)]


To compare the result with the starting point, we need to implement the contraction of the MPS. In python it is straightforward doing this with einsum.

In [6]:
def contract_MPS(mps):
    einstr = ("abc, cde, efg, ghi, ijk, klm")
    return oe.contract(einstr, *mps).squeeze()

If we contract v_MPS, reshape it back into the vector shape and compare it to the initial v, we see that it has not changed along the way. Thats what we wanted!

In [7]:
v_contracted = contract_MPS(v_MPS)
v_contracted = v_contracted.reshape(vector_shape)

np.allclose(
    v,
    v_contracted
)

True

Does this still hold, if we compress during the decomposition?

In [8]:
max_bond = int(2**(n/2)-2)
v_compressed = MPS(v_reshaped, max_bond=max_bond)
v_compressed = contract_MPS(v_compressed).reshape(vector_shape)

print("Are v and v_compressed allclose?     ", np.allclose(v, v_compressed))
print("Maximum difference:                  ", np. max(v - v_compressed))

Are v and v_compressed allclose?      False
Maximum difference:                   0.10897062572889898


## Hadamrd Multiplication
We realize elementwise multiplication using $n$ Kronecker delta tensors of 3rd order. For each site the Kronecker delta is contracted with the respective cores of the MPSs.

In [9]:
kron = np.zeros((2,2,2))
kron[0,0,0] = 1
kron[1,1,1] = 1


def multiply_MPS_MPS(kron, mps1, mps2):
    out = []
    for i in range(len(mps1)):
        shape_left = mps1[i].shape[0] * mps2[i].shape[0]
        shape_phys = kron.shape[0]
        shape_right = mps1[i].shape[2] * mps2[i].shape[2]
        out += [
            oe.contract("abc, dbf, gci -> dgafi", kron, mps1[i], mps2[i]).reshape(
                shape_left, shape_phys, shape_right
            )
        ]
    return out

Check again, if it works:

In [10]:
v_by_w = multiply_MPS_MPS(kron, v_MPS, w_MPS)
v_by_w = contract_MPS(v_by_w).reshape(vector_shape)

np.allclose(
    v*w,
    v_by_w
)

True

## Addition


In [11]:
def add_MPS_MPS(mps1, mps2):
    # first Block Matrix of shape
    # [A_1, B_1]
    out = [
        np.zeros(
            (
                1,
                mps1[0].shape[1],
                mps1[0].shape[2] + mps2[0].shape[2],
            )
        )
    ]
    out[0][:, :, : mps1[0].shape[2]] = mps1[0]
    out[0][:, :, mps1[0].shape[2] :] = mps2[0]

    # middle block matrices
    # [A_i,  0 ]
    # [ 0 , B_i]
    for i in range(1, len(mps1) - 1):
        out += [
            np.zeros(
                (
                    mps1[i].shape[0] + mps2[i].shape[0],
                    mps1[i].shape[1],
                    mps1[i].shape[2] + mps2[i].shape[2],
                )
            )
        ]
        out[-1][: mps1[i].shape[0], :, : mps1[i].shape[2]] = mps1[i]
        out[-1][mps1[i].shape[0] :, :, mps1[i].shape[2] :] = mps2[i]
    
    # last block matrix
    # [A_n]
    # [B_n]
    out += [
        np.zeros(
            (
                mps1[-1].shape[0] + mps2[-1].shape[0],
                mps1[-1].shape[1],
                1,
            )
        )
    ]
    out[-1][: mps1[-1].shape[0], :, :] = mps1[-1]
    out[-1][mps1[-1].shape[0] :, :, :] = mps2[-1]
    return out

In [12]:
v_plus_w = add_MPS_MPS(v_MPS, w_MPS)
v_plus_w = contract_MPS(v_plus_w).reshape(vector_shape)

np.allclose(
    v+w,
    v_plus_w
)

True

## MPO action
We start by generating a random Matrix A. Similar to the MPS case we reshape that matrix into binary indices.

In [13]:
A = np.random.random((2**n, 2**n))
A_reshaped = A.reshape((2,2)*n)

To keep track of which "ingoing" and "outgoing" indicies belong together, we reorder the binary indices. The new order has binary indicies of related to the same power of two next to each other.

In [35]:
MPO_ordering = ()
for i in range(n):
    MPO_ordering += (i,n+i)
print(MPO_ordering)

A_reordered = A_reshaped.transpose(MPO_ordering)

(0, 6, 1, 7, 2, 8, 3, 9, 4, 10, 5, 11)


#### MPO decomposition
By combining neighbouring indices again, we can reuse the MPS decomposition.

In [36]:
A_MPO = MPS(A_reordered.reshape((4,)*n))                 # combining neighbouring binary indices
for i, core in enumerate(A_MPO):
    shape = core.shape
    A_MPO[i] = core.reshape(shape[0],2,2,shape[-1])     # splitting in and outgoing indices
    print(A_MPO[i].shape)

(1, 2, 2, 4)
(4, 2, 2, 16)
(16, 2, 2, 64)
(64, 2, 2, 16)
(16, 2, 2, 4)
(4, 2, 2, 1)


#### Applying an MPO

Applying an MPO to an MPS is straightforward:
* contract the ingoing index with the respective mps physical index
* reshape core to merge bond indices

In [37]:
def apply_MPO(mpo, mps):
    out = []
    for i in range(len(mpo)):
        shape_left = mpo[i].shape[0] * mps[i].shape[0]
        shape_phys = mpo[i].shape[1]
        shape_right = mpo[i].shape[3] * mps[i].shape[2]
        out += [np.einsum("abcd, ecg -> aebdg", mpo[i], mps[i])]        # contraction
        out[-1] = out[-1].reshape(shape_left, shape_phys, shape_right)  # merging
    return out

#### Check

In [38]:
A_v_via_MPO = apply_MPO(A_MPO, v_MPS)
A_v_via_MPO = contract_MPS(A_v_via_MPO).reshape(vector_shape)

np.allclose(
    A_v_via_MPO,
    A@v
)

True